In [1]:
import cv2
import os
import numpy as np

# Define input and output directories
input_base_folder = "Dataset"
output_base_folder = "Preprocessed"

# Create output base directory if not exists
os.makedirs(output_base_folder, exist_ok=True)

# Target size for deep learning models
target_size = (224, 224)

# Loop through each category (Prasanna_Vijayakumar, others)
for category in os.listdir(input_base_folder):
    input_folder = os.path.join(input_base_folder, category)
    output_folder = os.path.join(output_base_folder, category)

    # Create category-specific output folder
    os.makedirs(output_folder, exist_ok=True)

    # Process each image
    for img_name in os.listdir(input_folder):
        img_path = os.path.join(input_folder, img_name)

        # Read the image
        img = cv2.imread(img_path)

        if img is None:
            print(f"❌ Skipping {img_name} (Unable to load)")
            continue  # Skip unreadable images

        # Convert to RGB
        img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

        # Resize to target size
        img_resized = cv2.resize(img_rgb, target_size)

        # Normalize pixel values (scale between 0-1)
        img_normalized = img_resized / 255.0

        # Save the preprocessed image (as .jpg)
        save_path = os.path.join(output_folder, img_name)
        cv2.imwrite(save_path, (img_normalized * 255).astype(np.uint8))

        # Optional: Save as numpy array for direct model input
        npy_save_path = os.path.join(output_folder, img_name.split('.')[0] + ".npy")
        np.save(npy_save_path, img_normalized)

    print(f"✅ Preprocessing complete for '{category}'! Saved in '{output_folder}'")

print("🎉 All images have been preprocessed successfully!")


✅ Preprocessing complete for 'others'! Saved in 'Preprocessed\others'
✅ Preprocessing complete for 'Prasanna_Vijayakumar'! Saved in 'Preprocessed\Prasanna_Vijayakumar'
🎉 All images have been preprocessed successfully!


In [11]:
import cv2
import os
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator, img_to_array
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.optimizers import Adam

# Define paths
input_base_folder = "Preprocessed"  # Folder with preprocessed images
image_size = (224, 224)  # Size to which the images were resized

# Load images and labels
images = []
labels = []

# Loop through each category (Prasanna_Vijayakumar, others)
for category in os.listdir(input_base_folder):
    input_folder = os.path.join(input_base_folder, category)
    
    for img_name in os.listdir(input_folder):
        img_path = os.path.join(input_folder, img_name)
        
        # Read image
        img = cv2.imread(img_path)
        if img is None:
            continue  # Skip if the image is not loaded
        
        # Resize image
        img = cv2.resize(img, image_size)
        
        # Convert to array and normalize
        img_array = img_to_array(img) / 255.0
        
        # Append the image to the list
        images.append(img_array)
        
        # Label the images
        if category == "Prasanna_Vijayakumar":
            labels.append(0)  # Label 0 for 'Prasanna_Vijayakumar'
        else:
            labels.append(1)  # Label 1 for 'Others'

# Convert to numpy arrays
X = np.array(images)
y = np.array(labels)

# Split into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Define ImageDataGenerator for data augmentation (for "Others" class)
train_datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

val_datagen = ImageDataGenerator()  # No augmentation for validation set

# Apply the data augmentation to the training set
train_generator = train_datagen.flow(X_train, y_train, batch_size=32)
val_generator = val_datagen.flow(X_val, y_val, batch_size=32)

# Define the CNN model
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(224, 224, 3)),
    MaxPooling2D(pool_size=(2, 2)),
    
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),
    
    Flatten(),
    
    Dense(128, activation='relu'),
    Dense(1, activation='sigmoid')  # Output layer for binary classification
])

# Compile the model
model.compile(optimizer=Adam(), loss='binary_crossentropy', metrics=['accuracy'])

# Train the model for 25 epochs
model.fit(
    train_generator,
    epochs=25,
    validation_data=val_generator
)

# Save the model
model.save('face_recognition_model_with_data_augmentation.h5')

print("Model training complete and saved as 'face_recognition_model_with_data_augmentation.h5'")


Epoch 1/25
25/25 [==============================] - 27s 1s/step - loss: 0.9308 - accuracy: 0.8255 - val_loss: 0.0577 - val_accuracy: 0.9949
Epoch 2/25
25/25 [==============================] - 24s 969ms/step - loss: 0.0817 - accuracy: 0.9785 - val_loss: 0.1570 - val_accuracy: 0.9444
Epoch 3/25
25/25 [==============================] - 24s 945ms/step - loss: 0.0938 - accuracy: 0.9709 - val_loss: 0.0128 - val_accuracy: 1.0000
Epoch 4/25
25/25 [==============================] - 24s 960ms/step - loss: 0.0469 - accuracy: 0.9848 - val_loss: 0.0222 - val_accuracy: 0.9899
Epoch 5/25
25/25 [==============================] - 24s 955ms/step - loss: 0.0393 - accuracy: 0.9874 - val_loss: 0.0014 - val_accuracy: 1.0000
Epoch 6/25
25/25 [==============================] - 24s 947ms/step - loss: 0.0221 - accuracy: 0.9899 - val_loss: 7.4949e-04 - val_accuracy: 1.0000
Epoch 7/25
25/25 [==============================] - 24s 972ms/step - loss: 0.0135 - accuracy: 0.9962 - val_loss: 0.0027 - val_accuracy: 1.000

In [32]:
import cv2
import numpy as np
from keras.models import load_model

# Load the pre-trained model
model = load_model('face_recognition_model_with_data_augmentation.h5')

# Load the Haar Cascade for face detection (OpenCV's pre-trained model)
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

# Define a function to preprocess the face image
def preprocess_face(face):
    target_size = (224, 224)  # Same as model input size
    face_resized = cv2.resize(face, target_size)
    face_normalized = face_resized / 255.0
    face_normalized = np.expand_dims(face_normalized, axis=0)  # Add batch dimension
    return face_normalized

# Start video capture (webcam)
cap = cv2.VideoCapture(1)

while True:
    # Capture frame by frame
    ret, frame = cap.read()
    
    # Convert the frame to grayscale for face detection
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    
    # Detect faces in the image
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))
    
    for (x, y, w, h) in faces:
        # Draw a rectangle around the face
        cv2.rectangle(frame, (x, y), (x + w, y + h), (255, 0, 0), 2)
        
        # Extract the face region from the frame
        face = frame[y:y + h, x:x + w]
        
        # Preprocess the face image
        face_preprocessed = preprocess_face(face)
        
        # Predict the label (Prasanna_Vijayakumar or Others)
        prediction = model.predict(face_preprocessed)
        
        # Determine the label
        if prediction[0][0] < 0.9:
            label = "Prasanna_Vijayakumar"
        else:
            label = "Others"
        
        # Display the label above the face
        cv2.putText(frame, label, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2)
    
    # Display the resulting frame
    cv2.imshow('Face Detection', frame)
    
    # Press 'q' to exit the loop
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the capture and close all OpenCV windows
cap.release()
cv2.destroyAllWindows()


In [ ]:
import cv2
import numpy as np
import pandas as pd
from keras.models import load_model
from datetime import datetime
import time

# Load the pre-trained model
model = load_model('face_recognition_model_with_data_augmentation.h5')

# Load the Haar Cascade for face detection (OpenCV's pre-trained model)
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

# Define a function to preprocess the face image
def preprocess_face(face):
    target_size = (224, 224)  # Same as model input size
    face_resized = cv2.resize(face, target_size)
    face_normalized = face_resized / 255.0
    face_normalized = np.expand_dims(face_normalized, axis=0)  # Add batch dimension
    return face_normalized

# Function to log face details into a CSV file
def log_face_details(name, time_entered, time_existed, duration):
    date_entered = time_entered.strftime("%Y-%m-%d")
    time_entered_str = time_entered.strftime("%H:%M:%S")
    time_existed_str = time_existed.strftime("%H:%M:%S")
    
    # Calculate duration in seconds or minutes
    duration_in_minutes = duration / 60
    duration_in_seconds = duration
    
    # Prepare the log data
    log_data = {
        "Name": name,
        "Date Entered": date_entered,
        "Time Entered": time_entered_str,
        "Time Existed": time_existed_str,
        "Duration (Seconds)": duration_in_seconds,
        "Duration (Minutes)": duration_in_minutes
    }
    
    # Create or append to CSV file using pd.concat()
    csv_file = "face_log.csv"
    try:
        df = pd.read_csv(csv_file)
        # Use pd.concat to append new data to the existing DataFrame
        df = pd.concat([df, pd.DataFrame([log_data])], ignore_index=True)
    except FileNotFoundError:
        df = pd.DataFrame([log_data])
    
    # Save the updated log data to CSV
    df.to_csv(csv_file, index=False)
    print(f"Log entry added: {log_data}")

# Start video capture (webcam)
cap = cv2.VideoCapture(1)

# Initialize a variable to keep track of the detected face's state
face_detected = False
face_entry_time = None

while True:
    # Capture frame by frame
    ret, frame = cap.read()
    
    # Convert the frame to grayscale for face detection
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    
    # Detect faces in the image
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))
    
    # Check if a face is detected
    if len(faces) > 0:
        # Start or continue tracking the detected face
        if not face_detected:
            face_detected = True
            face_entry_time = datetime.now()  # Capture the entry time when face is first detected
        
        # Draw a rectangle around the face
        for (x, y, w, h) in faces:
            cv2.rectangle(frame, (x, y), (x + w, y + h), (255, 0, 0), 2)
        
        # Extract the face region from the frame
        face = frame[y:y + h, x:x + w]
        
        # Preprocess the face image
        face_preprocessed = preprocess_face(face)
        
        # Predict the label (Prasanna_Vijayakumar or Others)
        prediction = model.predict(face_preprocessed)
        
        # Determine the label
        if prediction[0][0] < 0.8:
            label = "Prasanna_Vijayakumar"
        else:
            label = "Others"
        
        # Display the label above the face
        cv2.putText(frame, label, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2)
    
    else:
        # If no face is detected and a face was previously detected, log the exit time
        if face_detected:
            face_detected = False
            face_exit_time = datetime.now()  # Capture the exit time when face is no longer detected
            duration = (face_exit_time - face_entry_time).total_seconds()  # Calculate duration in seconds
            log_face_details(label, face_entry_time, face_exit_time, duration)
            print(f"Face exited: Duration: {duration} seconds")
    
    # Display the resulting frame
    cv2.imshow('Face Detection', frame)
    
    # Press 'q' to exit the loop
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the capture and close all OpenCV windows
cap.release()
cv2.destroyAllWindows()

Log entry added: {'Name': 'Others', 'Date Entered': '2025-02-10', 'Time Entered': '18:31:44', 'Time Existed': '18:31:45', 'Duration (Seconds)': 0.173194, 'Duration (Minutes)': 0.0028865666666666664}
Face exited: Duration: 0.173194 seconds
Log entry added: {'Name': 'Others', 'Date Entered': '2025-02-10', 'Time Entered': '18:31:46', 'Time Existed': '18:31:46', 'Duration (Seconds)': 0.194774, 'Duration (Minutes)': 0.0032462333333333334}
Face exited: Duration: 0.194774 seconds
Log entry added: {'Name': 'Others', 'Date Entered': '2025-02-10', 'Time Entered': '18:31:46', 'Time Existed': '18:31:46', 'Duration (Seconds)': 0.180144, 'Duration (Minutes)': 0.0030024}
Face exited: Duration: 0.180144 seconds
Log entry added: {'Name': 'Others', 'Date Entered': '2025-02-10', 'Time Entered': '18:31:47', 'Time Existed': '18:31:49', 'Duration (Seconds)': 2.424796, 'Duration (Minutes)': 0.04041326666666667}
Face exited: Duration: 2.424796 seconds
Log entry added: {'Name': 'Others', 'Date Entered': '2025-

Prediction (Raw output): [[1.]]
